# <center>The Battle of the Neighbourhood</center>

## Introduction
The battle of the neighbourhood is a capstone project for IBM Data Science Professional Cetrified for Data Science.

Question :

Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?

I have given my self the question of which region would be suited to set up an air conditioner companies in a region within New Zealand.  This will be based on comparision between the regions of Canterbury.

##  The business problem.

Which location in the two region have area lack converage for heat pump companies compare with PM10 data?

Which of the location have increasing incomes?  Leading to greater ability in purchasing heat pump as opposed to cheaper forms of heating requiring burning 
 
 ## Method

Software : 
Python 3.9.2 
JuptyerNotebook
Visual Studio Code


Data: 
I will be using PM10 from ECAN (Environment Canturbury).  Export data from Stats NZ and FourSquare API for Miscellous.



In [111]:
# Import the required libraries
import pandas as pd
import requests
import bs4 as soup
import io
import folium


Extract PM10 Data

PM10 is a fine particle that is created mainly from burning fuel sources.  In New Zealand most of the home heating is solid fuel heating woodburner(woodstove).  This is changing however due the stricter standards on air quality for city/town to prevent negative health effects.  

ECAN is the regulatory body for the province of Canturbury.  It task with monitoring and in forcing environmental standards.  As part of this it has a number of PM10 monitoring in it region 

https://www.ecan.govt.nz/







In [112]:
# ECAN Air Quality sites
ECAN_extract= []
ECANSiteList =   [7,2,9,4,5,1,10,64,77,3]
for Site in ECANSiteList:
    Extraction_Bay = requests.get('http://data.ecan.govt.nz/data/98/Air/Air%20quality%20data%20for%20a%20monitored%20site%20(Daily)/JSON?SiteId='+str(Site)+'&StartDate=01%2F01%2F2020&EndDate=01%2F01%2F2021')
    ECAN_extract.append(Extraction_Bay.json())



In [113]:
AirList = ['item',['DateTime']]

ECAN_list = []
for i in ECAN_extract:
    bay = i['data']['item'].copy()
    ECAN_list = ECAN_list+bay
    



ECANRaw = pd.json_normalize(ECAN_list)
ECAN = ECANRaw[['DateTime','StationName','PM10_x0020__x0028_ug_x002F_m3_x0029_']]
ECAN.columns = ['DateTime','StationName','PM10']
ECAN = ECAN[ECAN['PM10'].notnull()].copy()
ECAN['PM10'] = ECAN.PM10.astype(float).round(1)
ECAN.head()

,DateTime,StationName,PM10
0,2020-01-01,Ashburton,23.7
1,2020-01-02,Ashburton,18.8
2,2020-01-03,Ashburton,9.7
3,2020-01-04,Ashburton,11.5
4,2020-01-05,Ashburton,26.0


Foursquare is location based data repository.  It stored data relating to venues and check-in of customers which can build a real-time map of customer movements.

URL : https://foursquare.com/



In [114]:
CLIENT_ID = 'BBRUYDTJVTG2IYAVIKCHLF1GPKO1WGVTZKNFZJGLQFJKLLRQ' # your Foursquare ID
CLIENT_SECRET = 'GWHGWMRKZT5TS1UNTRAMKESJXDMTCIBJUIDCSF4DDIR5O1IY' # your Foursquare Secret
ACCESS_TOKEN = "VPMLHSDIANMBWYCWV4CI04RSLCAPIGLJR1GRSPTKPPF2ECT0" # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30



In [115]:
Location = pd.unique(ECAN.StationName)
print(Location)



['Ashburton' 'Geraldine' 'Kaiapoi' 'Rangiora' 'St Albans'
 'Timaru Anzac Square' 'Waimate Kennedy' 'Washdyke Alpine' 'Woolston']


In [116]:
Dict = {'Location': ['Ashburton', 'Geraldine', 'Kaiapoi', 'Rangiora', 'St Albans',
       'Timaru Anzac Square', 'Waimate Kennedy', 'Washdyke Alpine',
       'Woolston'], 
       'Lat': [-43.9054,-44.0923,-43.3788,-43.3032,-43.5320,-44.3904,-44.7327,-44.3539,-43.5473],
       'Long' : [171.7448,171.2447,172.6623,172.5966,172.6306,171.2373,171.0481,171.2379,172.6791]}
LatLongdf = pd.DataFrame(data= Dict)




In [117]:
Town_Lat = LatLongdf.loc[4,'Lat']
Town_Long = LatLongdf.loc[4,'Long']
Town_Name = LatLongdf.loc[4,'Location']

print('Latitude and longitude values of {} are {}, {}.'.format(Town_Name, 
                                                               Town_Lat, 
                                                               Town_Long))


Latitude and longitude values of St Albans are -43.532, 172.6306.


In [118]:
radius = 10000
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Town_Lat, 
    Town_Long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=BBRUYDTJVTG2IYAVIKCHLF1GPKO1WGVTZKNFZJGLQFJKLLRQ&client_secret=GWHGWMRKZT5TS1UNTRAMKESJXDMTCIBJUIDCSF4DDIR5O1IY&v=20180604&ll=-43.532,172.6306&radius=10000&limit=100'

In [119]:
results = requests.get(url).json()


In [120]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [121]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Christchurch Botanic Gardens,Botanical Garden,-43.531092,172.626088
1,Christchurch Art Gallery,Art Gallery,-43.530479,172.631072
2,North Hagley Park,Park,-43.528019,172.622495
3,YHA Christchurch,Hostel,-43.532189,172.629866
4,Bacon Brothers Burgers,Burger Joint,-43.535845,172.640867


In [122]:
#print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
Uniquecategory =pd.unique(nearby_venues.categories)
print(Uniquecategory)

['Botanical Garden' 'Art Gallery' 'Park' 'Hostel' 'Burger Joint'
 'Cheese Shop' 'Video Store' 'Arts & Crafts Store' 'Gym' 'History Museum'
 'Coffee Shop' 'Café' 'Food Court' 'Lounge' 'Bar' 'Playground' 'Pub'
 'Italian Restaurant' 'Gourmet Shop' 'Food Truck' 'Plaza' 'Bakery'
 'Supermarket' 'Asian Restaurant' 'Farmers Market' 'Japanese Restaurant'
 'Indian Restaurant' 'Dessert Shop' 'Gym / Fitness Center'
 'Breakfast Spot' 'Shopping Mall' 'Electronics Store' 'Stadium'
 'Modern European Restaurant' 'Chinese Restaurant' 'Boutique'
 'Flea Market' 'Pizza Place' 'Wine Bar' 'Basketball Stadium'
 'Mexican Restaurant' 'Beach' 'Gastropub' 'Department Store'
 'Convenience Store' 'Multiplex' 'Restaurant' 'Airport Lounge' 'Motel'
 'Museum' 'Thai Restaurant' 'Hotel' 'Scenic Lookout']


Above is  the search the largest location, which is Christchurch.   There is no information regarding home heating or construction supplies in aspects of venues.  I think this due the lack of the knowledge of the Foursquare application, especially due to the name being the same a supermarket in the country.


Foursquare is a dead end for the infomation the require to solved my question regarding the best place to set a home heating contracting business.





In [123]:
import 

SyntaxError: invalid syntax (Temp/ipykernel_20292/2494350585.py, line 1)